# Environment and Simulation Setup
## Initial Component Placement for the Simulation
This task may be trivial, but nonetheless necessary for the execution of the project’s other tasks. If the Jenga tower is initially unassembled, this task can also include the assembly process.

In [1]:
import os
import pydot

from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer
from pydrake.all import Simulator, StartMeshcat, DiagramBuilder, MeshcatVisualizer

from manipulation import running_as_notebook
from manipulation.station import MakeHardwareStation, load_scenario

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7006


In [3]:
def make_stack():
    output = ''
    num_layers = 18 # actual number is 18, but use a smaller number to speed up sim setup
    num_blocks_per_layer = 3
    block_width = 0.025
    block_height = 0.015
    for i in range(num_layers):
        # alternate block orientations
        layer_in_x_direction = i % 2 == 0
        for j in range(num_blocks_per_layer):
            x = (block_width * (j - 1)) if layer_in_x_direction else 0
            y = 0 if layer_in_x_direction else (block_width * (j - 1))
            z = block_height * (i + 0.5)
            orientation = "{ deg: [0, 0, 0] }" if layer_in_x_direction else "{ deg: [0, 0, 90] }"
            output += f"""
- add_model:
    name: block_{i * 3 + j}
    file: package://jengabot/models/jenga_block.sdf
    default_free_body_pose:
        jenga_block_link:
            translation: [{x}, {y}, {z}]
            rotation: !Rpy {orientation}"""
    return output

In [ ]:
def create_scene():
    scenario_data = """
directives:
- add_model:
    name: table_top
    file: package://jengabot/models/table_top.sdf
- add_weld:
    parent: world
    child: table_top::table_top_link
"""

    camera_data = """
    - add_frame:
        name: camera0_origin
        X_PF:
            base_frame: world
            rotation: !Rpy { deg: [0, 0, 0]}
            translation: [0, 1, 1]
    cameras:
        camera0:
        name: camera0
        depth: True
        X_PB:
            base_frame: camera0_origin
    """

    scenario_data += make_stack()
    # scenario_data += camera_data
    print(scenario_data)
    
    scenario = load_scenario(data=scenario_data)
    builder = DiagramBuilder()
    
    station = MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml'])
    #station = builder.AddSystem(MakeHardwareStation(scenario, meshcat, package_xmls=['./package.xml']))
    #context = station.CreateDefaultContext()


    simulator = Simulator(station)

    
    
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


create_scene()